
# 3. Model Comparison : Comparaison de différentes architectures CNN pour MIAS

## 1. Introduction

Ce notebook a pour objectif de comparer plusieurs architectures de réseaux de neurones convolutifs (CNN) pour la classification des mammographies du dataset MIAS.

### Objectifs :
- Implémenter un modèle baseline simple
- Tester un modèle CNN optimisé
- Expérimenter le transfert d’apprentissage (ResNet, etc.)
- Évaluer et comparer les performances de chaque modèle (accuracy, recall, F1-score)

## 2. Importation des librairies

Nous importons les librairies nécessaires pour la manipulation des données, la construction des modèles CNN et l’évaluation des performances.

In [14]:
# ----- Import libraries PEP 8 -----
# ----- Standard library -----
import numpy as np # Pour les opérations mathématiques et les tableaux 
import pandas as pd # Pour manipuler et analyser les métadonnées
import matplotlib.pyplot as plt # Pour les visualisations 
import tensorflow as tf # Pour le deep learning et la création de modèles
from tensorflow.keras import layers, models, applications
from sklearn.metrics import classification_report, confusion_matrix # Pour le rapport de classification
from sklearn.preprocessing import LabelEncoder
import pickle

print("Librairies importées avec succès.")

Librairies importées avec succès.


## 3. Chargement des données

Les données doivent être préparées comme dans le notebook précédent.  
On charge ici les ensembles d’entraînement et de test (images et labels).

In [15]:
X_train = np.load("../data/processed/X_train.npy") # Charge les images d'entraînement depuis le fichier numpy
X_test = np.load("../data/processed/X_test.npy")   # Charge les images de test depuis le fichier numpy
y_train = np.load("../data/processed/y_train.npy") # Charge les labels d'entraînement depuis le fichier numpy
y_test = np.load("../data/processed/y_test.npy")   # Charge les labels de test depuis le fichier numpy

print(f"Train shape : {X_train.shape}, Test shape : {X_test.shape}") # Affiche la forme des ensembles d'entraînement et de test
print(f"Train labels : {np.unique(y_train)}, Test labels : {np.unique(y_test)}") # Affiche les classes présentes dans chaque ensemble

Train shape : (264, 128, 128), Test shape : (66, 128, 128)
Train labels : ['ARCH' 'ASYM' 'CALC' 'CIRC' 'MISC' 'NORM' 'SPIC'], Test labels : ['ARCH' 'ASYM' 'CALC' 'CIRC' 'MISC' 'NORM' 'SPIC']


In [16]:
# Encodage des labels en entiers pour la classification multi-classe

le = LabelEncoder()
y_train = le.fit_transform(y_train) # Transforme les labels d'entraînement en entiers
y_test = le.transform(y_test) # Transforme les labels de test en entiers
print(f"Labels encodés : {np.unique(y_train)}") # Affiche les labels encodés

Labels encodés : [0 1 2 3 4 5 6]


## 4. Définition des architectures de modèles

Nous allons définir trois architectures :
- Un modèle baseline simple (CNN classique)
- Un modèle CNN optimisé (plus de couches, dropout)
- Un modèle utilisant le transfert d’apprentissage (ResNet50)

Chaque modèle sera entraîné et évalué sur le même jeu de données pour une comparaison équitable.

### 4.1 Modèle baseline simple (CNN classique)

Ce modèle sert de référence : il est composé de quelques couches de convolution et de pooling, suivi d’une couche dense pour la classification.

In [17]:
def build_simple_cnn(input_shape, num_classes): # input_shape : tuple qui définit la forme des images en entrée (ex : (128, 128, 1)), num_classes : entier, nombre de classes à prédire
    model = tf.keras.Sequential([ # création d'un modèle séquentiel Keras, qui empile les couches les unes sur les autres
        # --- Couche d'entrée ---
        tf.keras.layers.Input(shape=input_shape), # # couche d'entrée qui définit la forme la forme attendue pour chaque image

        # --- Couche intermédiaire ---
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu'), # Première couche de convolution : 32 : nombre de filtres (sorties), (3, 3) : taille du filtre (hauteur, largeur)
        tf.keras.layers.MaxPooling2D(2, 2), # Couche de sous-échantillonnage : réduit la taille de moitié
        tf.keras.layers.Flatten(), # Transforme la matrice 2D en un vecteur 1D (pour la couche dense)
        tf.keras.layers.Dense(64, activation='relu'), # Couche dense avec 64 neurones et activation ReLU

        # --- Couche de sortie ---
        tf.keras.layers.Dense(num_classes, activation='softmax') # Couche de sortie avec un neurone par classe et activation softmax
    ])
    return model

### 4.2 Modèle CNN optimisé

Ce modèle ajoute une couche de convolution supplémentaire, du dropout pour limiter l’overfitting, et une couche dense plus large.

In [18]:
def build_optimized_cnn(input_shape, num_classes): # input_shape : tuple (hauteur, largeur, canaux), num_classes : nombre de classes à prédire
    model = tf.keras.Sequential([ # Création d'un modèle séquentiel Keras
        # --- Couche d'entrée ---
        tf.keras.layers.Input(shape=input_shape), # Couche d'entrée, définit la forme attendue des images

        # --- Couche intermédiaire ---
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu'), # 1ère couche de convolution : 32 filtres, taille 3x3, activation ReLU
        tf.keras.layers.MaxPooling2D(2, 2), # 1er pooling : réduit la taille de moitié
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'), # 2ème couche de convolution : 64 filtres, taille 3x3, activation ReLU
        tf.keras.layers.MaxPooling2D(2, 2), # 2ème pooling : réduit la taille de moitié
        tf.keras.layers.Dropout(0.3), # Dropout : désactive 30% des neurones pour limiter l'overfitting
        tf.keras.layers.Flatten(), # Transforme la matrice 2D en vecteur 1D
        tf.keras.layers.Dense(128, activation='relu'), # Couche dense : 128 neurones, activation ReLU
        tf.keras.layers.Dropout(0.3), # Dropout : désactive 30% des neurones

        # --- Couche de sortie ---
        tf.keras.layers.Dense(num_classes, activation='softmax') # Couche de sortie : un neurone par classe, activation softmax
    ])
    return model # Retourne le modèle construit

### 4.3 Modèle avec transfert d’apprentissage (ResNet50)

Ce modèle utilise l’architecture ResNet50, connue pour ses très bonnes performances en vision par ordinateur.  
On adapte la sortie pour la classification MIAS et on ajoute une couche dense pour enrichir l’apprentissage.

In [19]:
def build_resnet_model(input_shape, num_classes): # input_shape : tuple (hauteur, largeur, canaux), num_classes : nombre de classes à prédire
    base_model = tf.keras.applications.ResNet50( # Charge le modèle ResNet50
        weights=None, # Pas de poids pré-entraînés (mettre 'imagenet' pour utiliser les poids ImageNet)
        include_top=False, # Ne garde pas la couche de classification finale de ResNet50
        input_shape=input_shape # Définit la forme attendue des images
    )
    
    x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output) # Applique un pooling global pour réduire la dimension
    x = tf.keras.layers.Dense(128, activation='relu')(x) # Couche dense de 128 neurones avec activation ReLU
    output = tf.keras.layers.Dense(num_classes, activation='softmax')(x) # Couche de sortie : un neurone par classe, activation softmax
    model = tf.keras.Model(inputs=base_model.input, outputs=output) # Crée le modèle final en reliant l'entrée de ResNet50 à la sortie personnalisée
    return model # Retourne le modèle construit

## 5. Entraînement et comparaison des modèles

Ce bloc de code entraîne les trois modèles (baseline, optimisé, ResNet) et sauvegarde leurs historiques pour une analyse ultérieure.

### 5.1 Préparation des données pour les modèles CNN

Avant d'entraîner les modèles, il est nécessaire d'ajouter une dimension canal aux images (pour Conv2D) et de définir la forme d'entrée et le nombre de classes. Cette étape garantit que les données sont compatibles avec les architectures Keras utilisées dans ce notebook.

In [20]:
X_train_cnn = X_train[..., np.newaxis] # Ajoute une dimension canal aux images d'entraînement (niveaux de gris -> (batch, height, width, 1))
X_test_cnn = X_test[..., np.newaxis]   # Ajoute une dimension canal aux images de test
input_shape = X_train_cnn.shape[1:]    # Récupère la forme des images (hauteur, largeur, canaux) pour l'entrée du modèle
num_classes = len(np.unique(y_train))  # Calcule le nombre de classes à prédire

### 5.2 Création et compilation du modèle baseline

On construit ici le modèle CNN de base (architecture simple) et on le compile avec l'optimiseur Adam et la fonction de perte adaptée à la classification multi-classe. Cette étape prépare le modèle pour l'entraînement.

In [21]:
model_baseline = build_simple_cnn(input_shape, num_classes) # Initialise le modèle CNN baseline avec la forme d'entrée et le nombre de classes
model_baseline.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy']) # Compile le modèle avec Adam et la fonction de perte adaptée à la classification multi-classe

### 5.3 Entraînement et sauvegarde du modèle baseline

On entraîne ici le modèle baseline sur les données d'entraînement, puis on sauvegarde l'historique d'entraînement (accuracy, loss) dans un fichier .pkl et le modèle lui-même au format .keras pour une utilisation ultérieure.

In [22]:
history_baseline = model_baseline.fit(X_train_cnn, y_train, epochs=10, batch_size=32, validation_split=0.2) # Entraîne le modèle baseline sur les données d'entraînement (10 époques, batch de 32, 20% validation)

with open("../models/history_baseline.pkl", "wb") as f: # Ouvre/crée le fichier pour sauvegarder l'historique d'entraînement
    pickle.dump(history_baseline.history, f) # Sauvegarde le dictionnaire d'historique (accuracy, loss, etc.) dans le fichier .pkl
model_baseline.save("../models/model_baseline.keras") # Sauvegarde le modèle entraîné au format .keras pour une utilisation ultérieure

Epoch 1/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 253ms/step - accuracy: 0.3555 - loss: 3.6292 - val_accuracy: 0.6038 - val_loss: 1.6330
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 253ms/step - accuracy: 0.3555 - loss: 3.6292 - val_accuracy: 0.6038 - val_loss: 1.6330
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 172ms/step - accuracy: 0.6019 - loss: 1.5054 - val_accuracy: 0.6604 - val_loss: 1.1486
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 172ms/step - accuracy: 0.6019 - loss: 1.5054 - val_accuracy: 0.6604 - val_loss: 1.1486
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 179ms/step - accuracy: 0.6209 - loss: 1.2852 - val_accuracy: 0.6415 - val_loss: 1.2050
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 179ms/step - accuracy: 0.6209 - loss: 1.2852 - val_accuracy: 0.6415 - val_loss: 1.2050
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 173ms/step - accuracy: 0.6351 - loss: 1.2391 - val_accuracy: 0.6981 - val_loss: 1.0866
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 173ms/step - accuracy: 0.6351 - loss: 1.2391 - val_accuracy: 0.6981 - val_loss:

### 5.4 Création et compilation du modèle CNN optimisé

On construit ici le modèle CNN optimisé (architecture plus complexe avec plus de couches et du dropout) et on le compile avec l'optimiseur Adam et la fonction de perte adaptée à la classification multi-classe.

In [23]:
model_optimized = build_optimized_cnn(input_shape, num_classes) # Initialise le modèle CNN optimisé avec la forme d'entrée et le nombre de classes
model_optimized.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy']) # Compile le modèle avec Adam et la fonction de perte adaptée à la classification multi-classe

### 5.5 Entraînement et sauvegarde du modèle optimisé

On entraîne ici le modèle CNN optimisé sur les données d'entraînement, puis on sauvegarde l'historique d'entraînement (accuracy, loss) dans un fichier .pkl et le modèle lui-même au format .keras pour une utilisation ultérieure.

In [24]:
history_optimized = model_optimized.fit(X_train_cnn, y_train, epochs=10, batch_size=32, validation_split=0.2) # Entraîne le modèle optimisé sur les données d'entraînement (10 époques, batch de 32, 20% validation)

with open("../models/history_optimized.pkl", "wb") as f: # Ouvre/crée le fichier pour sauvegarder l'historique d'entraînement
    pickle.dump(history_optimized.history, f) # Sauvegarde le dictionnaire d'historique (accuracy, loss, etc.) dans le fichier .pkl
model_optimized.save("../models/model_optimized.keras") # Sauvegarde le modèle entraîné au format .keras pour une utilisation ultérieure

Epoch 1/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 324ms/step - accuracy: 0.5261 - loss: 1.6027 - val_accuracy: 0.6792 - val_loss: 1.2367
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 324ms/step - accuracy: 0.5261 - loss: 1.6027 - val_accuracy: 0.6792 - val_loss: 1.2367
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 260ms/step - accuracy: 0.6161 - loss: 1.3487 - val_accuracy: 0.6792 - val_loss: 1.1865
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 260ms/step - accuracy: 0.6161 - loss: 1.3487 - val_accuracy: 0.6792 - val_loss: 1.1865
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 264ms/step - accuracy: 0.6161 - loss: 1.2457 - val_accuracy: 0.6981 - val_loss: 1.1952
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 264ms/step - accuracy: 0.6161 - loss: 1.2457 - val_accuracy: 0.6981 - val_loss: 1.1952
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 340ms/step - accuracy: 0.6256 - loss: 1.1791 - val_accuracy: 0.6981 - val_loss: 1.1348
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 340ms/step - accuracy: 0.6256 - loss: 1.1791 - val_accuracy: 0.6981 - val_loss:

### 5.6 Création et compilation du modèle ResNet50

On construit ici le modèle ResNet50 (transfert d'apprentissage) et on le compile avec l'optimiseur Adam et la fonction de perte adaptée à la classification multi-classe.

In [25]:
model_resnet = build_resnet_model(input_shape, num_classes) # Initialise le modèle ResNet50 avec la forme d'entrée et le nombre de classes
model_resnet.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy']) # Compile le modèle avec Adam et la fonction de perte adaptée à la classification multi-classe

### 5.7 Entraînement et sauvegarde du modèle ResNet50

On entraîne ici le modèle ResNet50 sur les données d'entraînement, puis on sauvegarde l'historique d'entraînement (accuracy, loss) dans un fichier .pkl et le modèle lui-même au format .keras pour une utilisation ultérieure.

In [26]:
history_resnet = model_resnet.fit(X_train_cnn, y_train, epochs=10, batch_size=32, validation_split=0.2) # Entraîne le modèle ResNet50 sur les données d'entraînement (10 époques, batch de 32, 20% validation)

with open("../models/history_resnet.pkl", "wb") as f: # Ouvre/crée le fichier pour sauvegarder l'historique d'entraînement
    pickle.dump(history_resnet.history, f) # Sauvegarde le dictionnaire d'historique (accuracy, loss, etc.) dans le fichier .pkl
model_resnet.save("../models/model_resnet.keras") # Sauvegarde le modèle entraîné au format .keras pour une utilisation ultérieure

Epoch 1/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 56s 3s/step - accuracy: 0.4028 - loss: 3.4579 - val_accuracy: 0.6792 - val_loss: 1.8104
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 56s 3s/step - accuracy: 0.4028 - loss: 3.4579 - val_accuracy: 0.6792 - val_loss: 1.8104
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 20s 3s/step - accuracy: 0.6114 - loss: 1.6914 - val_accuracy: 0.6792 - val_loss: 1.8385
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 20s 3s/step - accuracy: 0.6114 - loss: 1.6914 - val_accuracy: 0.6792 - val_loss: 1.8385
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 21s 3s/step - accuracy: 0.5877 - loss: 1.5967 - val_accuracy: 0.6792 - val_loss: 1.7350
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 21s 3s/step - accuracy: 0.5877 - loss: 1.5967 - val_accuracy: 0.6792 - val_loss: 1.7350
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 23s 3s/step - accuracy: 0.6019 - loss: 1.4872 - val_accuracy: 0.6792 - val_loss: 1.7197
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 23s 3s/step - accuracy: 0.6019 - loss: 1.4872 - val_accuracy: 0.6792 - val_loss: 1.7197
Epoch 5/

## 6. Conclusion

Dans ce notebook, nous avons comparé plusieurs architectures CNN pour la classification des mammographies MIAS :  
- Un modèle baseline simple  
- Un modèle CNN optimisé  
- Un modèle utilisant le transfert d'apprentissage (ResNet50)

Chaque modèle a été entraîné et évalué sur le même jeu de données.  
Les modèles et leurs historiques d'entraînement ont été sauvegardés pour une analyse ultérieure.  
Les résultats bruts (accuracy, loss) sont prêts à être analysés plus en détail dans le prochain notebook.

Le notebook suivant présentera une analyse approfondie des performances, des matrices de confusion et des rapports de classification pour interpréter les résultats et identifier le meilleur modèle pour cette tâche de classification.